In [1]:
pip install fedml-aws --force-reinstall

DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
Processing ./fedml_aws-2.0.0-py3-none-any.whl
  Using cached numpy-1.22.3-cp39-cp39-macosx_10_14_x86_64.whl (17.6 MB)
  Using cached pandas-1.4.2-cp39-cp39-macosx_10_9_x86_64.whl (11.1 MB)
  Using cached requests-2.27.1-py2.py3-none-any.whl (63 kB)
  Using cached hdbcli-2.12.20-cp34-abi3-macosx_10_7_x86_64.whl (5.6 MB)
  Using cached PyYAML-6.0-cp39-cp39-macosx_10_9_x86_64.whl (197 kB)
  Using cached pytz-2022.1-py2.py3-none-any.whl (503 kB)
  Using cached python_dateutil-2.8.2-py2.py3-none-any.whl (247 kB)
  Using cached charset_normalizer-2.0.12-py3-none-any.whl (39 kB)
  Using cached certifi-2021.10.8-py2.py3-none-any.whl (149 kB)
  Using cached urllib3-1.26.9-py2.py3-none-any.whl (138 kB)
  Using cached idna-3.3-py3-none-any.w

In [2]:
pip install boto3

DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
You should consider upgrading via the '/usr/local/opt/python@3.9/bin/python3.9 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [3]:
# please follow https://docs.aws.amazon.com/cli/latest/userguide/getting-started-install.html to install awscli

In [4]:
from fedml_aws import DbConnection
from fedml_aws import DwcSagemaker
import numpy as np
import pandas as pd

In [5]:
db = DbConnection()
res, column_headers = db.get_data_with_headers(table_name='SALES_VIEW', size=0.00001)
test_data= pd.DataFrame(res, columns=column_headers)
print(test_data.shape)

(62, 12)


In [6]:
test_data.dtypes

Region             object
Country            object
Order_ID           object
Item_Type          object
Sales_Channel      object
Order_Priority     object
Units_Sold          int64
Unit_Price        float64
Unit_Cost         float64
Total_Revenue     float64
Total_Cost        float64
Total_Profit      float64
dtype: object

In [7]:
df = test_data.iloc[:,6:-1]
df

,Units_Sold,Unit_Price,Unit_Cost,Total_Revenue,Total_Cost
0,7216,47.45,31.79,342399.20,229396.64
1,3049,47.45,31.79,144675.05,96927.71
2,1519,47.45,31.79,72076.55,48289.01
3,6909,47.45,31.79,327832.05,219637.11
4,6088,47.45,31.79,288875.60,193537.52
...,...,...,...,...,...
57,428,47.45,31.79,20308.60,13606.12
58,8804,47.45,31.79,417749.80,279879.16
59,4985,47.45,31.79,236538.25,158473.15
60,6106,47.45,31.79,289729.70,194109.74


In [8]:
type(df)

pandas.core.frame.DataFrame

In [9]:
!aws configure set aws_access_key_id '<aws_access_key_id>'
!aws configure set aws_secret_access_key '<aws_secret_access_key>'
!aws configure set region '<region>'

In [10]:
dwcs = DwcSagemaker()

2022-04-04 12:16:18,298: fedml_aws.logger INFO: Got region us-east-1 


In [11]:
r = dwcs.predict(endpoint_name='linear-v2', 
             body=df.to_csv(header=False, index=False).encode('utf-8'), 
             content_type='text/csv')

In [12]:
r

'[113002.56000000128, 47747.3400000025, 23787.540000002977, 108194.9400000014, 95338.08000000165, 56235.06000000237, 150727.50000000064, 149615.64000000065, 56798.82000000235, 79458.84000000195, 140360.58000000077, 10868.040000003217, 81948.78000000189, 124669.26000000112, 85268.70000000185, 73273.14000000202, 125859.42000000106, 17586.180000003085, 40261.86000000268, 19559.340000003052, 85832.46000000185, 103747.50000000146, 120488.04000000114, 133063.02000000095, 43848.000000002605, 129476.880000001, 65129.940000002214, 142662.60000000073, 10946.340000003216, 122993.64000000109, 64018.08000000221, 57628.80000000233, 137588.76000000088, 57002.40000000235, 53400.60000000242, 79176.96000000197, 40809.960000002655, 77141.16000000198, 107646.8400000014, 108210.6000000014, 22064.940000003007, 234.90000000340922, 94523.76000000167, 99519.30000000156, 104029.38000000143, 115210.6200000013, 35172.360000002765, 150163.74000000063, 22393.800000003008, 145168.2000000007, 15660.000000003121, 6207

In [13]:
test_data['Total_Profit']

0     113002.56
1      47747.34
2      23787.54
3     108194.94
4      95338.08
        ...    
57      6702.48
58    137870.64
59     78065.10
60     95619.96
61    140940.00
Name: Total_Profit, Length: 62, dtype: float64